In [ ]:
import sys

sys.path.append(f"..")
sys.path.append(r"../pmsat-inference")
from evaluation.utils import print_results_info
from charts import *

In [ ]:
# results_dir = r"../learning_results_34"  # first results with non-mat, only random new data
# results_dir = r"../learning_results_52"  # non-mat results, replay and cex
# results_dir = r"../learning_results_66"  # combined algo first run
# results_dir = r"../learning_results_110"  # first run new alg name system
# results_dir = r"../learning_results_114" # many results 5-8 (TO 10 min)
# results_dir = r"../learning_results_122" # many results 9-12 (TO 20 min)
# results_dir = r"../learning_results_123" # many results 9-12 (TO 30 min), sws2/3
# results_dir = r"../learning_results_124" # many results 9-12 (TO 60 min), sws3/5, no wcp, el 5 once
results_dir = r"../learning_results_129" # many many results with random walks (no glitches) -a "APMSL(ONLY_RW)" "APMSL(RW)" "APMSL" -o "Random" "None" -n 3 -ns 3-8 -ni 3 -no 3 --learn_num_times 3 

results = load_results(results_dir)
print_results_info(results)

In [ ]:
from collections import defaultdict

learned_incorrectly = [r for r in results if not r["learned_correctly"] and is_valid_result(r)]

reasons = defaultdict(int)
for r in learned_incorrectly:
    if r["timed_out"]:
        reasons["timed out"] += 1
        continue
    
    if r["learned_automaton_size"] is None:
        reasons["no hypothesis"] += 1
        continue
        
    if r["learned_automaton_size"] < r["original_automaton_size"]:
        reasons["too small"] += 1
        continue
        
    if r["learned_automaton_size"] > r["original_automaton_size"]:
        reasons["too big"] += 1
        
    reasons["too little data (probably)"] += 1
    
dict(reasons)

In [ ]:
bar_chart_per_algorithm_and_oracle(results, "learned_correctly", stat_method=np.mean, only_if=is_valid_result)
bar_chart_per_algorithm_and_oracle(results, "steps_learning", stat_method=np.mean, only_if=is_valid_result)
bar_chart_per_algorithm_and_oracle(results, "queries_learning", stat_method=np.mean, only_if=is_valid_result)
bar_chart_per_algorithm_and_oracle(results, "learning_rounds", stat_method=np.mean, only_if=is_valid_result)

# line_chart_over_learning_rounds("sliding_window_start")
# line_chart_over_learning_rounds("num_traces")
# line_chart_over_learning_rounds("num_glitches")
# line_chart_over_learning_rounds("preprocessing_additional_traces")
# line_chart_over_learning_rounds("postprocessing_additional_traces_cex")
# line_chart_over_learning_rounds("postprocessing_additional_traces_glitch")

# scatterplot_per_alg("learning_rounds", "queries_learning")
# scatterplot_per_alg("learning_rounds", "steps_learning")

bar_chart_per_algorithm_and_oracle(results, "total_time", stat_method=np.median, only_if=is_valid_result)
# bar_chart_per_algorithm(("last_pmsat_info", "num_vars"), stat_method=np.median, only_valid=True, only_if=lambda res: res.get("abort_reason", None) is None)
# bar_chart_per_algorithm(("last_pmsat_info", "num_hard"), stat_method=np.median, only_valid=True, only_if=lambda res: res.get("abort_reason", None) is None)
# bar_chart_per_algorithm(("last_pmsat_info", "num_soft"), stat_method=np.median, only_valid=True, only_if=lambda res: res.get("abort_reason", None) is None)

In [ ]:
num_removed_traces = sum(sum(r["detailed_learning_info"][str(i+1)].get("removed_traces", 0) for i in range(r["learning_rounds"])) for r in results)
print(f"Number of removed traces: {num_removed_traces}")

In [ ]:
for stat in ('learned_correctly', 'bisimilar', 'Precision', 'Recall', 'F-Score', 'Precision (all steps)', 'Precision (all traces)', 'Strong accuracy (mean)', 'Strong accuracy (median)', 'Medium accuracy (mean)', 'Medium accuracy (median)', 'Weak accuracy (mean)', 'Weak accuracy (median)'):
    bar_chart_per_algorithm_and_oracle(results, stat, stat_method=np.mean, only_if=is_valid_result)

In [ ]:
for stat in ('learned_correctly', 'steps_learning'):
    bar_chart_per_algorithm_and_oracle(results, stat, stat_method=np.mean, only_if=is_valid_result)

In [ ]:
for orig_num_states in sorted(list(set(r['original_automaton_size'] for r in valid_results))):
    print(orig_num_states)
    bar_chart_per_algorithm_and_oracle('learned_correctly', stat_method=np.mean, only_valid=True, only_if=lambda res: res['original_automaton_size'] == orig_num_states)
    bar_chart_per_algorithm_and_oracle('timed_out', stat_method=np.mean, only_valid=True, only_if=lambda res: res['original_automaton_size'] == orig_num_states)
    bar_chart_per_algorithm_and_oracle('Precision (all steps)', stat_method=np.mean, only_valid=True, only_if=lambda res: res['original_automaton_size'] == orig_num_states)
    bar_chart_per_algorithm_and_oracle('Precision (traces)', stat_method=np.mean, only_valid=True, only_if=lambda res: res['original_automaton_size'] == orig_num_states)

In [ ]:
print("test")

In [ ]:
bar_chart_per_algorithm_and_oracle('timed_out', stat_method=sum)